In [30]:
%load_ext autoreload
%autoreload 2

In [31]:
#! pip install einops

In [32]:
import sys,os
import torch
sys.path.append(os.path.abspath('../models'))
from tinyism import tinyModel

In [33]:
posencoding=True
model=tinyModel(posEncoding=posencoding)

In [34]:
 weight="../weights/resnet18_inst.pth"
 model.load_state_dict(torch.load(weight,map_location=torch.device('cpu')))
 model.eval()

tinyModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [50]:
    sys.path.append(os.path.abspath('../datasets'))
    from data_loader import preprocess
    import cv2
    img=cv2.imread("../test.png")
    img=cv2.resize(img,(512,512))
    x=preprocess((torch.from_numpy(img).float())).unsqueeze(0)

In [51]:
## ONNX conversion
y1,y2=model(x)
x.shape,y1.shape,y2.shape

(torch.Size([1, 3, 512, 512]), torch.Size([4]), torch.Size([4, 1, 64, 64]))

In [52]:
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "tinynet.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=12,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size',1: 'instances'},    # variable length axes
                                'output' : {0 : 'batch_size',1: 'outputinstances'}})

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/einops/einops.py:204: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  inferred_length: int = length // known_product
/home/ec2-user/SageMaker/tiny-instance-segmentation/models/tinyism.py:42: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  if len(bidx)>maxdets:
/home/ec2-user/SageMaker/tiny-instance-segmentation/models/tinyism.py:52: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect.

In [ ]:
## check conversion
!pip install onnxruntime

In [53]:
import onnxruntime
ort_session = onnxruntime.InferenceSession("tinynet.onnx")

In [54]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [55]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

In [69]:
input_image=to_numpy(x)
input_image.shape

(1, 3, 512, 512)

In [56]:
y1=to_numpy(y1)
y2=to_numpy(y2)
ort_outs[0].shape,ort_outs[1].shape,y1.shape,y2.shape

((4,), (4, 1, 64, 64), (4,), (4, 1, 64, 64))

In [76]:
ort_outs[1]

array([[[[0.16491759, 0.16338453, 0.16040626, ..., 0.16071782,
          0.16183221, 0.15289238],
         [0.16723716, 0.17236674, 0.17438   , ..., 0.16114116,
          0.16439745, 0.15792394],
         [0.16422993, 0.16862863, 0.17395005, ..., 0.16700858,
          0.16054675, 0.15998146],
         ...,
         [0.13980982, 0.15187079, 0.17807859, ..., 0.15225616,
          0.16712594, 0.17388237],
         [0.13712534, 0.14206952, 0.16376051, ..., 0.16094291,
          0.15684304, 0.15327564],
         [0.13647446, 0.142115  , 0.14452785, ..., 0.17303154,
          0.18076345, 0.16862842]]],


       [[[0.15834999, 0.15854937, 0.15511957, ..., 0.18060303,
          0.1717546 , 0.17188361],
         [0.16032788, 0.16409266, 0.16513076, ..., 0.18978205,
          0.18537393, 0.1771403 ],
         [0.16137469, 0.1650309 , 0.17318162, ..., 0.17538962,
          0.17245567, 0.1754936 ],
         ...,
         [0.13875446, 0.14308533, 0.14854527, ..., 0.14195853,
          0.11302778, 0

In [57]:
import numpy as np
np.allclose(y1, ort_outs[0], rtol=1e-03, atol=1e-05),np.allclose(y2, ort_outs[1], rtol=1e-03, atol=1e-05)

(True, True)

In [58]:
## Openvio conversion

In [16]:
!pip install openvino-dev
!pip install openvino-dev[onnx]

[autoreload of numpy.core.multiarray failed: Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/numpy/core/mult

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [18]:
! mo -h

usage: main.py [options]

optional arguments:
  -h, --help            show this help message and exit
  --framework {paddle,caffe,kaldi,mxnet,onnx,tf}
                        Name of the framework used to train the input model.

Framework-agnostic parameters:
  --input_model INPUT_MODEL, -w INPUT_MODEL, -m INPUT_MODEL
                        {} Tensorflow*: a file with a pre-trained model
                        (binary or text .pb file after freezing). Caffe*: a
                        model proto file with model weights
  --model_name MODEL_NAME, -n MODEL_NAME
                        Model_name parameter passed to the final create_ir
                        transform. This parameter is used to name a network in
                        a generated IR and output .xml/.bin files.
  --output_dir OUTPUT_DIR, -o OUTPUT_DIR
                        Directory that stores the generated IR. By default, it
                        is the directory from where the Model Optimizer is
               

In [59]:
from IPython.display import Markdown, display

In [97]:
# Construct the command for Model Optimizer.
from pathlib import Path
onnx_path="tinynet.onnx"
model_path="irmodel"
model_path = Path(model_path)
ir_path = model_path.with_suffix(".xml")
IMAGE_HEIGHT=512
IMAGE_WIDTH=512
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --input_shape "[1,3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
                 --data_type FP32
                 --output_dir "{model_path}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert the ONNX model to OpenVINO:


`mo --input_model "tinynet.onnx" --input_shape "[1,3, 512, 512]" --data_type FP32 --output_dir "irmodel"`

In [98]:
print("Exporting ONNX model to IR... This may take a few minutes.")
mo_result = %sx $mo_command
print("\n".join(mo_result))

Exporting ONNX model to IR... This may take a few minutes.
[ WARNING ]  Use of deprecated cli option --data_type detected. Option use in the following releases will be fatal. 
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ec2-user/SageMaker/tiny-instance-segmentation/notebooks/irmodel/tinynet.xml
[ SUCCESS ] BIN file: /home/ec2-user/SageMaker/tiny-instance-segmentation/notebooks/irmodel/tinynet.bin


In [99]:
ir_path=Path("irmodel/tinynet.xml")

In [100]:
from openvino.runtime import Core
ie = Core()

In [101]:
model_ir = ie.read_model(model=ir_path)
compiled_model_ir = ie.compile_model(model=model_ir, device_name="CPU")

In [102]:
compiled_model_ir

<CompiledModel:
inputs[
<ConstOutput: names[input] shape[1,3,512,512] type: f32>
]
outputs[
<ConstOutput: names[output] shape[..4096] type: i64>,
<ConstOutput: names[872] shape[4,1,64,64] type: f32>
]>

In [103]:
# Get input and output layers.
output_layer_ir = compiled_model_ir.output(1)
output_layer_ir

<ConstOutput: names[872] shape[4,1,64,64] type: f32>

In [104]:
res_ir = compiled_model_ir([input_image])[output_layer_ir]
res_ir.shape

(4, 1, 64, 64)

In [105]:
res_ir

array([[[[0.1649176 , 0.16338456, 0.16040629, ..., 0.16071784,
          0.16183224, 0.15289238],
         [0.16723719, 0.17236677, 0.17437999, ..., 0.16114116,
          0.16439743, 0.15792395],
         [0.16422993, 0.16862865, 0.1739501 , ..., 0.16700855,
          0.16054681, 0.15998141],
         ...,
         [0.13980986, 0.1518708 , 0.1780786 , ..., 0.15225607,
          0.16712587, 0.17388214],
         [0.13712543, 0.14206952, 0.1637606 , ..., 0.16094285,
          0.15684302, 0.15327564],
         [0.13647453, 0.14211504, 0.14452797, ..., 0.17303163,
          0.18076348, 0.1686283 ]]],


       [[[0.15835004, 0.15854935, 0.15511955, ..., 0.180603  ,
          0.17175457, 0.1718836 ],
         [0.1603279 , 0.16409267, 0.16513075, ..., 0.189782  ,
          0.18537395, 0.17714027],
         [0.16137466, 0.16503088, 0.17318162, ..., 0.1753896 ,
          0.17245564, 0.17549357],
         ...,
         [0.13875452, 0.14308529, 0.14854528, ..., 0.14195853,
          0.11302772, 0

In [106]:
y2-res_ir

array([[[[ 0.0000000e+00,  0.0000000e+00,  1.4901161e-08, ...,
           0.0000000e+00, -1.4901161e-08,  2.9802322e-08],
         [ 1.4901161e-08,  5.9604645e-08,  1.4901161e-08, ...,
           0.0000000e+00, -2.9802322e-08, -1.4901161e-08],
         [-1.4901161e-08,  4.4703484e-08, -1.4901161e-08, ...,
           1.4901161e-08,  2.9802322e-08, -4.4703484e-08],
         ...,
         [-4.4703484e-08, -4.4703484e-08, -1.4901161e-08, ...,
           7.4505806e-08,  1.4901161e-08,  8.9406967e-08],
         [-4.4703484e-08,  0.0000000e+00, -7.4505806e-08, ...,
           1.4901161e-08, -5.9604645e-08,  0.0000000e+00],
         [ 0.0000000e+00, -4.4703484e-08, -7.4505806e-08, ...,
          -4.4703484e-08, -1.4901161e-08,  0.0000000e+00]]],


       [[[ 0.0000000e+00,  2.9802322e-08,  7.4505806e-08, ...,
           2.9802322e-08,  1.4901161e-08, -2.9802322e-08],
         [-2.9802322e-08,  0.0000000e+00,  1.4901161e-08, ...,
           1.4901161e-08,  5.9604645e-08,  2.9802322e-08],
      

In [110]:
np.allclose(res_ir,y2,rtol=1e-04, atol=1e-05)

True

In [ ]:
## Weights conversion

In [ ]:
posencoding=True
model_curr=tinyModel(posEncoding=posencoding,deploy=False)
curr_state_dict=model_curr.state_dict()

In [ ]:
device=torch.device("cpu")
loaded_state_dict=torch.load('../weights/RepVGGplus-L2pse-train256-acc84.06.pth',map_location=device)
new_state_dict={}
matched_layers=0
unmatched_layers=0
for k in curr_state_dict.keys():
    if "backbone" in k:
        modk=k.split(".")[1:]
        modk=".".join(modk)
        #print(f"mod k is {modk}")
    else:
        modk=k
    if modk in loaded_state_dict.keys() and loaded_state_dict[modk].size()==curr_state_dict[k].size():
        new_state_dict[k]=loaded_state_dict[modk]
        matched_layers+=1
    else:
        new_state_dict[k]=curr_state_dict[k]
        unmatched_layers+=1

In [ ]:
## find any layer of same size
device=torch.device("cpu")
loaded_state_dict=torch.load('../weights/RepVGGplus-L2pse-train256-acc84.06.pth',map_location=device)
new_state_dict={}
matched_layers=0
unmatched_layers=0
for k in curr_state_dict.keys():
    findsz=curr_state_dict[k].size()
    found=False
    for kl in loaded_state_dict.keys():
        foundsz=loaded_state_dict[kl].size()
        if findsz==foundsz:
            new_state_dict[k]=loaded_state_dict[kl]
            matched_layers+=1
            found=True
    if not found:
        new_state_dict[k]=curr_state_dict[k]
        unmatched_layers+=1

In [ ]:
print(f"Matched Layers {matched_layers}")
print(f"Unmatched Layers {unmatched_layers}")

In [ ]:
#loaded_state_dict.keys()

In [ ]:
#curr_state_dict.keys()

In [ ]:
torch.save(new_state_dict, "../weights/repvgg_pretrain.pth")

In [ ]:
## Load repvggplus
import torch

In [ ]:
sys.path.append(os.path.abspath('../models'))
from repvggplus_tinysim import tinyModel

In [ ]:
posencoding=True
model_curr=tinyModel(posEncoding=posencoding,deploy=False)

In [ ]:
device=torch.device("cpu")
loaded_state_dict=torch.load('../weights/RepVGGplus-L2pse-train256-acc84.06.pth',map_location=device)

In [ ]:
model_curr.backbone

In [ ]:
model_curr.backbone.load_state_dict(loaded_state_dict,strict=False)